In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob 
from scipy.optimize import curve_fit

In [ ]:
# get data
# data should be in csv format and contain the following columns:
# Series (Timeseries ID), CellNr (Cell ID within the timeseries), X (Framenumber),
# Median_SubBackg (median intensity values after background subtraction),
# NormMedian (median intensity values, normalized to the value at the first timepoint)

df_ctrl = pd.read_csv("D:/work/Cohesin_Paper/Auxin_treatment_Timeseries/Measurements_series1to7.csv") 
df_treat = pd.read_csv("D:/work/Cohesin_Paper/Auxin_treatment_Timeseries/Measurements_series8to14.csv") 


# filter traces by mean square deviation from smoothed version
def rollmean_filter(df, window_size=5, cutoff=0.1, column='NormMedian', valid_column='Valid'):
    df_tot = []
    for s in pd.unique(df['Series'].values):
        series = df.loc[(df['Series'] == s)]
        for c in pd.unique(series['CellNr'].values):
            cell = series.loc[(series['CellNr'] == c)]
            
            # calculate rolling mean via convolution
            rm = np.convolve(cell[column].values, np.ones(window_size,)/window_size, mode='valid')
            # msd in valid portion
            msd = np.mean((cell[column].values[window_size//2:-(window_size//2)] - rm)**2)
            cell[valid_column] = msd < cutoff
            df_tot.append(cell)
    return pd.concat(df_tot)


# Filter traces by minimum intensity value
def rm_low_int(df, intensity=50):
    df_x1 = df[df['X']==1]
    df_int = df_x1[df_x1['Median_SubBackg']<=intensity]
    cellnr = np.array(df_int['CellNr'])
    seriesnr = np.array(df_int['Series'])
    for i in range(len(cellnr)):
        df_new = df.drop(df[(df['Series'] == seriesnr[i]) & (df['CellNr'] == cellnr[i])].index)
        df = df_new
    return df


df_treat = rollmean_filter(rm_low_int(df_treat), cutoff=0.00235) #cutoff = 90 % Quantile
df_ctrl = rollmean_filter(rm_low_int(df_ctrl), cutoff=0.0006) #cutoff = 90 % Quantile

df_treat_filt = df_treat[df_treat.Valid]
df_ctrl_filt = df_ctrl[df_ctrl.Valid]

In [ ]:
# Plot Control traces (6 hours) with mean trace and standard error
df_ctrl_filt['Minutes'] = (df_ctrl_filt['X']-1)*15 # Convert frame number to min

# plot the individual traces
plt.figure(figsize=(7,6))
for _, dfi in df_ctrl_filt.groupby(['CellNr', 'Series']):
    plt.plot(dfi.Minutes, dfi.NormMedian, color='lightgray', alpha=0.5)

# get mean and stderr per timepoint
means  = []
e = []
for _, dfi in df_ctrl_filt.groupby(['Minutes']):
    means.append(np.mean(dfi.NormMedian))
    e.append(dfi.NormMedian.std()/np.sqrt(len(dfi)))

# plot means with errorbars
x = list(range(0, len(means)))
x = [i * 15 for i in x]
plt.plot(x,means, color='black')
plt.title('Ctrl')
plt.errorbar(x, means, e, linestyle='None', marker='.', color='black')
_ = plt.ylim(0,1.2)
_ = plt.xlim(0,360)
_ = plt.xticks(np.arange(0, 361, step=60))

#plt.savefig('Ctrl_timeseries.pdf')  

In [ ]:
# Plot auxin treated traces (6 hours) with mean trace and standard error
df_treat_filt['Minutes'] = (df_treat_filt['X']-1)*15

# plot the individual traces
plt.figure(figsize=(7,6))
for _, dfi in df_treat_filt.groupby(['CellNr', 'Series']):
    plt.plot(dfi.Minutes, dfi.NormMedian, color='lightgray', alpha=0.5)

# get mean and stderr per timepoint
means  = []
e = []
for _, dfi in df_treat_filt.groupby(['Minutes']):
    means.append(np.mean(dfi.NormMedian))
    e.append(dfi.NormMedian.std()/np.sqrt(len(dfi)))

# plot means with errorbars
x = list(range(0, len(means)))
x = [i * 15 for i in x]
plt.plot(x,means, color='black')
plt.title('+Auxin')
plt.errorbar(x, means, e, linestyle='None', marker='.', color='black')
_ = plt.ylim(0,1.2)
_ = plt.xlim(0,360)
_ = plt.xticks(np.arange(0, 361, step=60))

#plt.savefig('Auxtreat_timeseries.pdf')  